In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler #scaling de los datos entre 0 y 1
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [3]:
#cargo primero los datos
file_dir = 'Datos/arg.txt'
data = pd.read_csv(file_dir,sep=",",quotechar='"',na_values=[''])
df = pd.DataFrame(data)
df = df.to_numpy()
df1 = np.copy(df) #el original
df = df[:-20] #saco los ultimos 20 dias (ult 10 para probar + ult 10 retrasados)
df1 = df1[:-10] #saco los ultimos 10 dias retrasados
#normalizacion
scaler = MinMaxScaler(feature_range=(0,1))
df = scaler.fit_transform(df.reshape(-1,1))
#split en train y test
training_size=int(len(df)*0.8)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1] #train y test data
lag = 3 #creo que este es el que deducen en el paper
#creo los que vienen para la red
x_train, y_train = create_dataset(train_data, lag)
x_test, y_test = create_dataset(test_data, lag)

In [4]:
# reshape en (samples, time steps, features) para lstm como en pract7
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1] , 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=100,activation='relu',input_shape=(lag,1),recurrent_dropout=0.5)))
model.add(keras.layers.Dense(units=1))
model.compile(optimizer='adam',loss=keras.losses.MSE,metrics=['mse']) #metrics=['mean_absolute_percentage_error']
history = model.fit(x_train, y_train,epochs=300,validation_data=(x_test,y_test),batch_size=4,verbose=2)

Train on 234 samples, validate on 56 samples
Epoch 1/300
 - 1s - loss: 0.0064 - mse: 0.0064 - val_loss: 0.0643 - val_mse: 0.0643
Epoch 2/300
 - 0s - loss: 2.6309e-04 - mse: 2.6309e-04 - val_loss: 0.0243 - val_mse: 0.0243
Epoch 3/300
 - 0s - loss: 1.4279e-04 - mse: 1.4279e-04 - val_loss: 0.0215 - val_mse: 0.0215
Epoch 4/300
 - 0s - loss: 1.0674e-04 - mse: 1.0674e-04 - val_loss: 0.0154 - val_mse: 0.0154
Epoch 5/300
 - 0s - loss: 5.7616e-05 - mse: 5.7616e-05 - val_loss: 0.0118 - val_mse: 0.0118
Epoch 6/300
 - 0s - loss: 5.7785e-05 - mse: 5.7785e-05 - val_loss: 0.0100 - val_mse: 0.0100
Epoch 7/300
 - 1s - loss: 4.4308e-05 - mse: 4.4308e-05 - val_loss: 0.0081 - val_mse: 0.0081
Epoch 8/300
 - 1s - loss: 3.2261e-05 - mse: 3.2261e-05 - val_loss: 0.0067 - val_mse: 0.0067
Epoch 9/300
 - 0s - loss: 2.6285e-05 - mse: 2.6285e-05 - val_loss: 0.0051 - val_mse: 0.0051
Epoch 10/300
 - 0s - loss: 1.9467e-05 - mse: 1.9467e-05 - val_loss: 0.0049 - val_mse: 0.0049
Epoch 11/300
 - 0s - loss: 2.5344e-05 - ms

 - 0s - loss: 7.8645e-06 - mse: 7.8645e-06 - val_loss: 1.7584e-04 - val_mse: 1.7584e-04
Epoch 85/300
 - 0s - loss: 5.0316e-06 - mse: 5.0316e-06 - val_loss: 3.2582e-05 - val_mse: 3.2582e-05
Epoch 86/300
 - 0s - loss: 4.8841e-06 - mse: 4.8841e-06 - val_loss: 9.9180e-05 - val_mse: 9.9180e-05
Epoch 87/300
 - 0s - loss: 7.8543e-06 - mse: 7.8543e-06 - val_loss: 1.4769e-05 - val_mse: 1.4769e-05
Epoch 88/300
 - 0s - loss: 1.0255e-05 - mse: 1.0255e-05 - val_loss: 1.8837e-05 - val_mse: 1.8837e-05
Epoch 89/300
 - 0s - loss: 5.0120e-06 - mse: 5.0120e-06 - val_loss: 1.3287e-05 - val_mse: 1.3287e-05
Epoch 90/300
 - 0s - loss: 2.7297e-06 - mse: 2.7297e-06 - val_loss: 8.4033e-05 - val_mse: 8.4033e-05
Epoch 91/300
 - 0s - loss: 6.3028e-06 - mse: 6.3028e-06 - val_loss: 5.8184e-05 - val_mse: 5.8184e-05
Epoch 92/300
 - 0s - loss: 4.1816e-06 - mse: 4.1816e-06 - val_loss: 8.7688e-06 - val_mse: 8.7688e-06
Epoch 93/300
 - 0s - loss: 2.7436e-06 - mse: 2.7436e-06 - val_loss: 2.9606e-05 - val_mse: 2.9606e-05
Epo

In [ ]:
train_predict=model.predict(x_train)
test_predict=model.predict(x_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
print('train mse squared:',np.sqrt(mean_squared_error(y_train,train_predict))) 
print('test mse squared:',np.sqrt(mean_squared_error(y_test,test_predict))) 

In [ ]:
trainPredictPlot = np.empty_like(df)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lag:len(train_predict)+lag, :] = train_predict
# shift test predictions for plotting
testPredictPlot = np.empty_like(df)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(lag*2)+1:len(df)-1, :] = test_predict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(df))
plt.plot(trainPredictPlot,label='train')
plt.plot(testPredictPlot,label='test')
plt.grid()
plt.title('Argentina')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
n = len(test_data)
x_input=test_data[n-lag:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
lst_output=[]
n_steps=lag
i=0

while(i<10): 
    if(len(temp_input)>lag):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, lag, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, lag,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i=i+1

In [ ]:
plt.plot(df1)
days = np.arange(len(df1))
dias_forecast = days[-10:]
plt.plot(dias_forecast,scaler.inverse_transform(lst_output))